<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Load data to Vantage
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Welcome to this introductory guide. This guide will walk you through steps  are needed to load data in Teradata from ipython SQL kernel.  A more extensive notebook with more commands can be found online in the Teradata Vantage Modules for Jupyter <a href = 'https://github.com/Teradata/jupyterextensions/blob/master/notebooks/sql/DataLoad.ipynb'>Here.</a>  

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For help on available "magic" commands in the Teradata SQL plugin, run the following cell:</p>

In [1]:
%help

%help **command** - Display detailed help for one magic command (without %).</br>
%help **all** - Display detailed help for all magic commands.</br>
%help **snippets** - Display the list of SQL command Snippets.</br>
%help **rowcount** - Display the current rowcount value.</br>
%help **currentdb** - Display the current database for the user.</br>
%help **hidewarnings** - Display the current hide warnings value.</br>
#### Notation conventions: Items in [] are optional, all others are required.  Parameter names are not case sensitive.</br>
#### Magic Commands:</br>
**%addconnect** - Add a Teradata Vantage database connection specification.</br>**%connect** - Connect to a Vantage system and/or make the connection active.</br>**%disconnect** - Disconnect from a Vantage system.</br>**%rmconnect** - Remove a connection specification.</br>**%chconnect** - Change a connection specification.</br>**%lsconnect** - List the known connections.</br>**%chart** - Display a Vega-Lite visualization of a query result set or an existing Vega or VegaLite chart.</br>**%table** - Display a previous result set as a table.</br>**%meta** - Display metadata of a previous result set.</br>**%dataload** - Load a CSV file into a database table.</br>**%history** - Display a table or the details of previously executed commands.
		If a single history item is specified, the details for that item will be displayed.</br>**%rmhistory** - Remove one or more history items and associated result set(s).</br>**%var** - Define one or more variables for use in SQL statements and magic commands.</br>**%lsvar** - List one or more variables. If no names are provided, list all defined variables.</br>**%rmvar** - Remove one or more defined variables.</br>**%pyinfo** - Display information about the version of Python installed on the Vantage system.</br>**%rinfo** - Display information about the version of R installed on the Vantage system.</br>**%runsto** - Run a Script Table Operator.</br>**%installfile** - Install a file onto the Vantage system.</br>**%rmfile** - Remove a file that has been installed on the Vantage system.</br>**%showfile** - Display the contents of a file that has been installed on the Vantage system.</br>**%loglevel** - Set the logging level for the Teradata SQL kernel.</br>**%lsmagic** - List the supported magic commands.</br>**%quickref** - Display help for magic commands.</br>**%help** - Display help for magic commands.</br>**%savecolumn** - Save a BLOB column data to a file.</br>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press Enter, then use down arrow to go to next cell.</p>

In [2]:
%connect local, hidewarnings=true

Password: ········


Success: 'local' connection established and activated for user 'demo_user', with default database 'demo_user'


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'> <b>2. Load data from file present locally </b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Apart from the standard Teradata utilities for data load i.e TPT; Teradata SQL plugin for Jupyter has a magic command to load data into the database.</p>

In [3]:
%help dataload

#### %dataload
Load a CSV file into a database table.

%dataload table=&lt;TableName&gt;, filepath=&lt;Path to CSV file&gt; [,database=&lt;DatabaseName&gt;][, batchsize=&lt;Rows per Batch&gt;] [, autocommit=&lt;True&gt;] [, mode=&lt;batch|fast&gt;] [, skiprows=&lt;rows to skip at the start of file&gt;]

* **database** is optional. if not specified, the default database of the current session is used.
* The **filepath** is relative to the JupyterLab workspace (where JupyterLab was launched).
* You can specify an absolute file path by starting the path with the OS specific directory notation.
* **batchsize** specifies the number of rows per batch.
  * Valid values are between 1 and 16383 for batch load. The default value is 5000 for batch load and 100000 for fast load.
* **autocommit** indicates whether to commit each batch load.
  * The default value is false (only commit when all batches have run). When autocommit is set to false and an error occurs, a rollback is performed.
  * When autocommit is set to true and an error occurs, any rows loaded will NOT be rolled back.
  * Set autocommit to true when a Database error is returned indicating the load has exceeded the maximum number of rowhash locks allowed.
  * autocommit is always turned off for fast load.
* **mode** specifies whether to use batch load or fast load. Fast load supports quick loading of large amounts of data.
  * It is recommended to use fast load only for large files with at least 100,000 rows.
  * The default mode is batch load for files with less than 100,000 rows. Fast load is used for large files with at least 100,000 rows.
  * The destination table must be empty to use fast load.
  * FastLoad cannot load into a volatile table or global temporary table.
  * Please make sure there is enough space in database before loading data. If you encounter "No more room in database" error using fast load, please make more space, drop and recreate the table, and then retry fast load.
  * For more information on fast load, please refer to github.com/Teradata/python-driver#fastload
* **skiprows** specifies number of rows to skip at start of the CSV file.
* The maximum number of rows that the **%dataload** magic will load is currently set to 100,000 rows for batch load and 500,000 for fast load.  Loading will stop when the maximum is reached.
* The data must be in the proper format to insert into the columns of your table. Specifically, data types must match and any data constraints must be met. 
No pre-processing of the data file is performed.
* If an error occurs while attempting to load your data and autocommit is false, the entire load operation will be rolled back. 
* The status of your dataload is stored in the Teradata SQL History.

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data we will be loading for this example is in CSV format. The following is a sample of the header followed by the first 5 rows:</p>

<p style = 'font-size:12px;font-family:Courier'>
InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country<br>
536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom<br>
536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom<br>
536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom<br>
536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom<br>
536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom<br>
</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will need to create a table that matches the columns of the data in our file.</p

In [4]:
CREATE MULTISET TABLE DEMO_USER.Retail_Data 
       (InvoiceNo VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        StockCode VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Description VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Quantity BIGINT,
        InvoiceDate TIMESTAMP(6),
        UnitPrice FLOAT,
        CustomerID FLOAT,
        Country VARCHAR(25) CHARACTER SET UNICODE NOT CASESPECIFIC)
  PRIMARY INDEX ( InvoiceNo );

Success: 0 rows affected

In [5]:
%dataload DATABASE = demo_user, TABLE = Retail_Data, skiprows = 1,
FILEPATH = Getting_Started/Data_Loading/Retail_Data_sample.csv

Using database: demo_user
2024-09-09T06:05:20.936 Start Batch Loading...
     
Data Load: Number Rows Loaded = 500, Elapsed Time = 00:00:00.345, History ID = 598
Success Loading Data: from file,'/home/jovyan/JupyterLabRoot/Getting_Started/Data_Loading/Retail_Data_sample.csv', into table, demo_user.Retail_Data.

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's check the count and sample records loaded. </p>


In [6]:
SELECT COUNT(*) FROM Retail_Data;

,Count(*)
1,500


In [7]:
SELECT TOP 5 * FROM Retail_Data;

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,536387,79321,CHILLI LIGHTS,192,2010-01-12 09:58:00.000000,3.819999933242798,16029,United Kingdom
2,536376,22114,HOT WATER BOTTLE TEA AND SYMPATHY,48,2010-01-12 09:32:00.000000,3.450000047683716,15291,United Kingdom
3,536376,21733,RED HANGING HEART T-LIGHT HOLDER,64,2010-01-12 09:32:00.000000,2.549999952316284,15291,United Kingdom
4,C536379,D,Discount,-1,2010-01-12 09:41:00.000000,27.5,14527,United Kingdom
5,536395,22188,BLACK HEART CARD HOLDER,8,2010-01-12 10:47:00.000000,3.950000047683716,13767,United Kingdom


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'> <b>3. Load data from file present in cloud </b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can access and load files present in cloud via Vantage's NOS (Native Object Store). For this we need the access id and key for authentication and the location off the file. Please refer <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Native-Object-Store-Getting-Started-Guide-17.20/Reading-Data/Examples-For-Business-Analysts/Sampling-External-Data-In-Place-Using-READ_NOS'>NOS user guide</a>  for more information.<br>
  First of all we need to create an Authorization object to access the file. </p> 

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The command template to create an authorization object is below. If you are loading data from your cloud storage, you would need to substitute for the values in the {  } brackets. </p>

<p style = 'font-size:12px;font-family:Courier'> CREATE AUTHORIZATION {name} AS DEFINER TRUSTED<br>
USER '{access_key}'<br>
PASSWORD '{password}';<br>
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>For public buckets we dont need any authorizationn. For our example we have a public read-only bucket hence we won't need authorization in queries below. </p>

<hr style="height:1px;border:none;background-color:#00233C;">

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1 Sampling External Data</b><br>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
Our file Retail_Data_sample.csv is located at /gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/<br>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can preview the contents of an external object store by using the command below. The location string is prefixed indicating where the data is located.
<ul>
    <li> "/gs" - to access files in google cloud storage, used below</li>
    <li> "/s3" - to access AWS buckets (Used in the UseCases/Deep_History_via_Object_Store)</li>
    <li> "/az" - to access Azure blob storage</li>
</ul>


In [8]:
SELECT TOP 2 * FROM (
LOCATION = '/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv'
) AS D;

,Location,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,/GS/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
2,/GS/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom


<p style = 'font-size:16px;font-family:Arial;color:#00233C'> Please keep in mind:
<ul style = 'font-size:16px;font-family:Arial;color:#00233C'><li>TOP does not return the same results each time it is run.</li>
<li>TOP may return records with only a subset of attributes because not all records in different objects contain all of the same attributes.</li>
    </ul>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.2 Using Foreign Table</b>

In [9]:
CREATE FOREIGN TABLE Retail_Data_FT
USING (LOCATION('/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv'));

Success: 0 rows affected

In [10]:
SELECT TOP 5 * FROM Retail_Data_FT;

,Location,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,/GS/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12 08:26:00,2.75,17850.0,United Kingdom
2,/GS/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
3,/GS/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
4,/GS/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv,536365,71053,WHITE METAL LANTERN,6,2010-01-12 08:26:00,3.39,17850.0,United Kingdom
5,/GS/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12 08:26:00,2.55,17850.0,United Kingdom


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.3 Loading External Data in permanent table</b><br>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>If we want to copy data from cloud to permanent table in database we can use the below command

In [11]:
CREATE MULTISET TABLE DEMO_USER.Retail_Data_Perm 
       (InvoiceNo VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        StockCode VARCHAR(10) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Description VARCHAR(40) CHARACTER SET UNICODE NOT CASESPECIFIC,
        Quantity BIGINT,
        InvoiceDate TIMESTAMP(6),
        UnitPrice FLOAT,
        CustomerID FLOAT,
        Country VARCHAR(25) CHARACTER SET UNICODE NOT CASESPECIFIC)
  PRIMARY INDEX ( InvoiceNo );

Success: 0 rows affected

In [12]:
INSERT DEMO_USER.Retail_Data_Perm 
SELECT
    InvoiceNo,
    StockCode,
    Description,
    Quantity,
    InvoiceDate,
    UnitPrice,
    CustomerID,
    Country
FROM (
    LOCATION = '/gs/storage.googleapis.com/clearscape_analytics_demo_data/Data_Load/Retail_Data_sample.csv'
) AS d;

Success: 500 rows affected

In [13]:
SELECT TOP 5 * FROM Retail_Data_Perm;

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,536376,21733,RED HANGING HEART T-LIGHT HOLDER,64,2010-01-12 09:32:00.000000,2.55,15291,United Kingdom
2,C536379,D,Discount,-1,2010-01-12 09:41:00.000000,27.5,14527,United Kingdom
3,536395,22867,HAND WARMER BIRD DESIGN,48,2010-01-12 10:47:00.000000,2.1,13767,United Kingdom
4,536387,22466,FAIRY TALE COTTAGE NIGHTLIGHT,432,2010-01-12 09:58:00.000000,1.45,16029,United Kingdom
5,536387,22780,LIGHT GARLAND BUTTERFILES PINK,192,2010-01-12 09:58:00.000000,3.37,16029,United Kingdom


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.4 Parquet Data on AWS</b><br>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>As mentioned previously, several of the demonstration notebooks access data on AWS. One of them is:</p>

[Deep History via Object Store](../../UseCases/Deep_History_via_Object_Store/Deep_History_via_Object_Store_SQL.ipynb)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data it accesses is in parquet format. Storage of data in the cloud is frequently broken up into multiple files to allow parallel reading of the data for performance and to allow some partitioning of the data using the directory hierarchy (in this case by year). The data is public so it can be accessed without an Authorization object. The data for that demo is in 28 files which we can view with the following command. </p>

In [14]:
SELECT TOP 5 location(char(255)), ObjectLength 
FROM read_nos (
USING 
 LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/') 
 RETURNTYPE ('NOSREAD_KEYS')
) AS d 
ORDER BY 1;

,Location,ObjectLength
1,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet,12484548
2,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2011/2011/object_33_0_1.parquet,16564887
3,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2011/2011/object_33_0_2.parquet,1242222
4,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2012/2012/object_33_2_1.parquet,16561265
5,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2012/2012/object_33_2_2.parquet,15091468


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The definition of the data can be read using the RETURNTYPE 'NOSREAD_PARQUET_SCHEMA'.   In this case we are only going to reference 1 file out of the set for performance.  We will specify FULLSCAN 'TRUE' to read through all rows to determine the maximum column length for varchar columns (if any ... at least for 1 file).  </p>

In [15]:
SELECT TOP 5 * FROM READ_NOS (
      USING
      LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet')
      RETURNTYPE ('NOSREAD_PARQUET_SCHEMA')
      FULLSCAN ('TRUE')
      )
AS d;

,ColPosition,Name,TdatDataType,PhysicalType,LogicalType,Precision,Scale,MinLength,MaxLength,NullFound,FileType,Location
1,3,sales_date,DATE,INT32,DATE,0,0,0,0,1,parquet,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet
2,5,store_id,INTEGER,INT32,NONE,0,0,0,0,1,parquet,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet
3,4,customer_id,INTEGER,INT32,NONE,0,0,0,0,1,parquet,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet
4,2,TheMonth,INTEGER,INT32,NONE,0,0,0,0,1,parquet,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet
5,1,TheYear,INTEGER,INT32,NONE,0,0,0,0,1,parquet,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2010/2010/object_33_0_1.parquet


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Again, to see a sample of the data, we will only reference one file.  If you remove "2010/2010/object_33_0_1.parquet" from the location below, it will run a bit longer.</p>

In [16]:
SELECT TOP 5 * FROM READ_NOS (
      USING
      LOCATION  ('/s3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/') --2010/2010/object_33_0_1.parquet')
      RETURNTYPE ('NOSREAD_RECORD')
      )
AS d;

,Location,TheYear,TheMonth,sales_date,customer_id,store_id,basket_id,product_id,sales_quantity,discount_amount
1,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2011/2011/object_33_0_1.parquet,2011,8,2011-08-10,2517,7,3625177,60,3,0.164221
2,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2013/2013/object_33_0_2.parquet,2013,11,2013-11-08,1599,9,31115999,23,3,0.0475902
3,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2013/2013/object_33_0_2.parquet,2013,2,2013-02-02,1599,9,22515999,8,2,0.183379
4,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2018/2018/object_33_0_7.parquet,2018,5,2018-05-11,4182,2,12741822,49,10,0.150882
5,/S3/s3.amazonaws.com/clearscape-analytics-datasets/sales_fact_offload/2018/2018/object_33_0_7.parquet,2018,5,2018-05-30,4182,2,10841822,35,7,0.0672583


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The creating of a foreign table or loading into a local table from Parquet on the AWS cloud is similar to the process described above.  For the full example, see the UseCase:</p>

[Deep History via Object Store](../../UseCases/Deep_History_via_Object_Store/Deep_History_via_Object_Store_SQL.ipynb)

 <p style = 'font-size:16px;font-family:Arial;color:#00233C'> To load other file formats like json or more details about parquet, please refer to: <a href ='https://docs.teradata.com/r/Enterprise_IntelliFlex_VMware/Teradata-VantageTM-Native-Object-Store-Getting-Started-Guide-17.20/Reading-Data/Examples-For-Business-Analysts/Sampling-External-Data-In-Place-Using-READ_NOS'>NOS user guide</a> </p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Cleanup</b>

In [17]:
DROP TABLE Retail_Data;

Success: 22 rows affected

In [18]:
DROP TABLE Retail_Data_FT;

Success: 31 rows affected

In [19]:
DROP TABLE Retail_Data_Perm;

Success: 22 rows affected

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>